- Ricardo Méndez 21289
- Sara Echeverría 21371
- Melissa Pérez 21385
- Francisco Castillo 21562

# Task 1

### ¿Por qué el modelo de Naive Bayes se le considera “naive”?

### Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? 

### ¿Qué tipo de ensemble learning es este modelo (Random Forest)?

### ¿Cuál es la idea general detrás de Random Forest?

### ¿Por qué se busca baja correlación entre los árboles de Random Forest?

# Task 2 Naive Bayes: Clasificador de Mensajes Ham/Spam

# Task 2.1

# Task 2.2

# Task 2.3

# Task 2.4

# Task 3 Clasificación de Partidas de League of Legends

# Task 3.1 Support Vector Machines

# Task 3.2 Árboles de Decisión

# Task 3.3 Comparación